In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [60]:
data_dir = "HERRERA_NATIVI_VLADIMIR"
metadata_Vlad = pd.read_csv("HERRERA_NATIVI_VLADIMIR/metadata_copy.csv")
metadata_Javi = pd.read_csv("PENA_CASTANO_JAVIER/metadata.csv")

# La prof ma parlé d'un tensorboard

In [61]:
print(f"Vladi : {metadata_Vlad.shape} | Javi : {metadata_Javi.shape}")

Vladi : (5627, 13) | Javi : (5477, 13)


In [62]:
# Fusionner les datasets
#metadata_combined = pd.concat([metadata_Vlad, metadata_Javi])

# Supprimer les doublons en gardant la première occurrence
#metadata_combined = metadata_combined.drop_duplicates(subset="Image Index", keep="first")

# Afficher un aperçu
#print(metadata_combined.head())

# Sauvegarder le dataset fusionné si besoin
#metadata_combined.to_csv("metadata_combined.csv", index=False)

#print(f"Merged  : {metadata_combined.shape}")

In [63]:
# Elimination des colonnes non necessaire pour notre étude initial
colonnes_a_eliminer = ['OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]']
data = metadata_Vlad.drop(columns=colonnes_a_eliminer, inplace=False) 
data

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,WEIGHTS,train
0,00000006_000.png,No Finding,0,6,81,M,PA,1,True
1,00000025_000.png,Effusion,0,25,71,M,PA,1,True
2,00000029_000.png,No Finding,0,29,59,F,PA,1,False
3,00000072_000.png,Atelectasis,0,72,67,F,PA,1,True
4,00000090_000.png,No Finding,0,90,67,F,PA,1,True
...,...,...,...,...,...,...,...,...,...
5622,00030752_000.png,No Finding,0,30752,64,F,AP,1,True
5623,00030772_000.png,No Finding,0,30772,26,F,AP,1,True
5624,00030772_001.png,Consolidation,1,30772,26,F,AP,1,True
5625,00030772_002.png,Consolidation,2,30772,26,F,AP,1,True


In [64]:
# Encodage des colonnes a valeurs non continue 

# Encodage binaire pour Patient Gender et View Position
data['Patient Gender'] = (data['Patient Gender'] == 'M').astype(int) # M est la réference et donc sera associé a la valeur 0 
data['View Position'] = (data['View Position'] == 'PA').astype(int) # PA est la réference et donc sera associé a la valeur 0  

# On effectue un encodage binaire pour finding labels du fait qu'on s'interesse uniquement au fait d'être malade ou pas (on n'essaye pas de prédire les maladies)
data['Finding Labels'] = (data['Finding Labels'] == 'No Finding').astype(int) # No Finding est la réference et donc sera associé a la valeur 0 (tous le reste des maladies seront mise a 1)

# dataset_encod.head()

In [65]:
data.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'WEIGHTS', 'train'],
      dtype='object')

In [66]:

# Sauvegarder une fois que le pretraitement sera fait 
#data.to_csv(f"{data_dir}/metadata_encod.csv", index=False) # sauvegarde de csv encodé 

In [67]:
dataset_col_utiles = data.loc[data['Patient Age'] <= 120] # On élimine les patients qui ont plus de 120 ans 

# Première methode naïve de pre-processing : reweighting

In [68]:

# -------------------------------------------  Répondération en fonction du genre  -------------------------------------------
nbtotSexe =  (data["Patient Gender"]).count()
nbFemmes = (data["Patient Gender"] == 1).sum()
nbHommes = (data["Patient Gender"] == 0).sum()
nbMalades = (data["Finding Labels"] == 0).sum()
nbSains= nbtotSexe - nbMalades


propH = nbHommes/nbtotSexe
propF = nbFemmes/nbtotSexe
propM = nbMalades/nbtotSexe
propS = 1 - propM

# études pour les femmes
nbFemmesMalades = ((data["Patient Gender"] == 1) & (data["Finding Labels"] == 1)).sum()
nbFemmesSain= nbFemmes - nbFemmesMalades
propFMF = nbFemmesMalades / nbFemmes
propFM = nbFemmesMalades / nbtotSexe
propFS = 1 - propFM
propFSF = 1 - propFMF

# étude pour les hommes
nbHommesMalades = ((data["Patient Gender"] == 0) & (data["Finding Labels"] == 1)).sum()
nbHommesSain = nbHommes - nbHommesMalades
propHMH = nbHommesMalades / nbHommes  # proportions des Hommes malades sur popHommes 
propHM = nbHommesMalades / nbtotSexe 
propHS = 1 - propHM
propHSH = 1 - propHMH  # proportions des Hommes sains sur popHommes 


poidsFM = (nbFemmes * nbMalades) / (nbFemmesMalades * nbtotSexe)
poidsFS = (nbFemmes * nbSains) / (nbFemmesSain * nbtotSexe)

poidsHM = (nbHommes * nbMalades) / (nbHommesMalades * nbtotSexe)
poidsHS = (nbHommes * nbSains) / (nbHommesSain * nbtotSexe)

print(f"Poids calculées pour les sexes :\nFemme Malade : {poidsFM}\nFemme Saine : {poidsFS}\nHomme Malade : {poidsHM}\nHomme Sain : {poidsHS}\n")

Poids calculées pour les sexes :
Femme Malade : 0.8652716513398885
Femme Saine : 1.1518640538396279
Homme Malade : 0.8241034868846114
Homme Sain : 1.2205941645286227



In [69]:
# ------------------------------------------- Répondération en fonction de la position du Rayon X -------------------------------------------
#  PA == 0

nbtotPosi =  (data["View Position"]).count()
nbAP = (data["View Position"] == 1).sum()
nbPA = (data["View Position"] == 0).sum()
nbMalades = (data["Finding Labels"] == 0).sum()
nbSains = nbtotPosi - nbMalades


propPA = nbPA/nbtotPosi
propAP = nbAP/nbtotPosi
propM = nbMalades/nbtotPosi
propS = 1 - propM

#nb de PA malades
nbAPMalades = ((data["View Position"] == 1) & (data["Finding Labels"] == 1)).sum()
nbAPSain= nbFemmes - nbFemmesMalades


#nb de AP malades
nbPAMalades = ((data["View Position"] == 0) &( data["Finding Labels"] == 1)).sum()
nbPASain = nbPA - nbPAMalades


poidsAPM = (nbAP * nbMalades) / (nbAPMalades * nbtotPosi)
poidsAPS = (nbAP * nbSains) / (nbAPSain * nbtotPosi)

poidsPAM = (nbPA * nbMalades) / (nbPAMalades * nbtotPosi)
poidsPAS = (nbPA * nbSains) / (nbPASain * nbtotPosi)

print(f"Poids calculées pour les positions :\nPA Malade : {poidsPAM}\nPA Sain : {poidsPAS}\nAp Malade : {poidsAPM}\nAP Sain : {poidsAPS}\n")

Poids calculées pour les positions :
PA Malade : 0.9741578820448354
PA Sain : 1.0229780041263805
Ap Malade : 0.7788565407014685
AP Sain : 1.2305741568542938



In [70]:
# ------------------------------------------- Répondération en fonction du 'Patient Age' -------------------------------------------
seuil_age = 40
data['Age Group'] = np.where(data['Patient Age'] >= seuil_age, 1, 0) # 1 répresent les "vieux"


nbtotAge = data['Age Group'].count()
nbVieux = (data['Age Group'] == 1).sum()
nbJeunes = (data['Age Group'] == 0).sum()

nbMalades = (data["Finding Labels"] == 0).sum()
nbSains = nbtotAge - nbMalades


propJeunes = nbJeunes/nbtotAge
propVieux = nbVieux/nbtotAge
propM = nbMalades/nbtotAge
propS = 1 - propM

#nb de PA malades
nbVieuxMalades = ((data["Age Group"] == 1) & (data["Finding Labels"] == 1)).sum()
nbVieuxSain= nbFemmes - nbFemmesMalades


#nb de AP malades
nbJeunesMalades = ((data["View Position"] == 0) &( data["Finding Labels"] == 1)).sum()
nbJeunesSain = nbJeunes - nbJeunesMalades


poidsVieuxM = (nbVieux * nbMalades) / (nbVieuxMalades * nbtotAge)
poidsVieuxS = (nbVieux * nbSains) / (nbVieuxSain * nbtotAge)

poidsJeunesM = (nbJeunes * nbMalades) / (nbJeunesMalades * nbtotAge)
poidsJeunesS = (nbJeunes * nbSains) / (nbJeunesSain * nbtotAge)

print(f"Poids calculées pour les groupes d'ages :\nJeune Malade : {poidsJeunesM}\nJeune Sain : {poidsJeunesS}\nVieux Malade : {poidsVieuxM}\nVieux Sain : {poidsVieuxS}\n")

Poids calculées pour les groupes d'ages :
Jeune Malade : 0.7780274284598085
Jeune Sain : 1.3185216168849088
Vieux Malade : 0.8574555435845033
Vieux Sain : 1.395646734010052



In [74]:
metadata_Vlad

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],WEIGHTS,train
0,00000006_000.png,No Finding,0,6,81,M,PA,2500,2048,0.168,0.168,1,True
1,00000025_000.png,Effusion,0,25,71,M,PA,2048,2500,0.171,0.171,1,True
2,00000029_000.png,No Finding,0,29,59,F,PA,2500,2048,0.171,0.171,1,False
3,00000072_000.png,Atelectasis,0,72,67,F,PA,2450,2839,0.143,0.143,1,True
4,00000090_000.png,No Finding,0,90,67,F,PA,2310,2991,0.143,0.143,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5622,00030752_000.png,No Finding,0,30752,64,F,AP,3056,2544,0.139,0.139,1,True
5623,00030772_000.png,No Finding,0,30772,26,F,AP,3056,2544,0.139,0.139,1,True
5624,00030772_001.png,Consolidation,1,30772,26,F,AP,3056,2544,0.139,0.139,1,True
5625,00030772_002.png,Consolidation,2,30772,26,F,AP,3056,2544,0.139,0.139,1,True


In [ ]:
# First, create the new columns with default values
data_c = metadata_Vlad.drop(columns=colonnes_a_eliminer, inplace=False) 
data_c['Age Group'] = np.where(data_c['Patient Age'] >= seuil_age, 1, 0)
data_c['poids_reweigth_gender'] = 1.0
data_c['poids_reweigth_PA'] = 1.0
data_c['poids_reweigth_age'] = 1.0

# Assign weights based on gender and health status
# For females (Gender == "F")
data_c.loc[(data_c['Patient Gender'] == "F") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_gender'] = poidsFM
data_c.loc[(data_c['Patient Gender'] == "F") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_gender'] = poidsFS

# For males (Gender == 0)
data_c.loc[(data_c['Patient Gender'] == "M") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_gender'] = poidsHM
data_c.loc[(data_c['Patient Gender'] == "M") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_gender'] = poidsHS

# Assign weights based on view position and health status
# For AP position (View Position == 1)
data_c.loc[(data_c['View Position'] == "AP") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_PA'] = poidsAPM
data_c.loc[(data_c['View Position'] == "AP") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_PA'] = poidsAPS

# For PA position (View Position == 0)
data_c.loc[(data_c['View Position'] == "PA") & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_PA'] = poidsPAM
data_c.loc[(data_c['View Position'] == "PA") & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_PA'] = poidsPAS

# Assign weights based on age and health status
# For older patients (Age Group == 1)
data_c.loc[(data_c['Age Group'] == 1) & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_age'] = poidsVieuxM
data_c.loc[(data_c['Age Group'] == 1) & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_age'] = poidsVieuxS

# For younger patients (Age Group == 0)
data_c.loc[(data_c['Age Group'] == 0) & (data_c['Finding Labels'] != "No Finding"), 'poids_reweigth_age'] = poidsJeunesM
data_c.loc[(data_c['Age Group'] == 0) & (data_c['Finding Labels'] == "No Finding"), 'poids_reweigth_age'] = poidsJeunesS

# Save the updated data_cset with the new columns
data_c.to_csv("HERRERA_NATIVI_VLADIMIR/metadata_c_with_weights.csv", index=False)

# Display a sample to verify the weights were added correctly
print(data_c[['Patient Gender', 'View Position', 'Age Group', 'Finding Labels', 
           'poids_reweigth_gender', 'poids_reweigth_PA', 'poids_reweigth_age']].head(10))

  Patient Gender View Position  Age Group                   Finding Labels  \
0              M            PA          1                       No Finding   
1              M            PA          1                         Effusion   
2              F            PA          1                       No Finding   
3              F            PA          1                      Atelectasis   
4              F            PA          1                       No Finding   
5              F            AP          1                       No Finding   
6              F            AP          1                       No Finding   
7              F            AP          1                         Effusion   
8              F            AP          1             Atelectasis|Effusion   
9              F            AP          1  Effusion|Emphysema|Infiltration   

   poids_reweigth_gender  poids_reweigth_PA  poids_reweigth_age  
0               1.220594           1.022978            1.395647  
1        